In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from dask.distributed import Client

from satpy import Scene, find_files_and_readers
from satpy.resample import get_area_def

from pyresample import create_area_def

from glob import glob

In [2]:
# Let's load Dask to make Xarray more efficient
import dask.distributed as dsk

# Start client
try:
    clnt
except:
    clnt = dsk.Client()
clnt

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 256.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44273,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 7
Started: Just now,Total memory: 256.00 GiB
Comm: tcp://127.0.0.1:40955,Total threads: 1
Dashboard: /proxy/43491/status,Memory: 36.57 GiB
Nanny: tcp://127.0.0.1:34313,


In [3]:
var = 'ACHA_mode_8_cloud_optical_depth_vis'#ACHA_mode_8_cloud_top_temperature'
day_range = range(53, 69)

out_path = '/g/data/hh5/tmp/WACI-Hackathon-2023/share/cr7888/AUS2200_CLEX_analysis/plots/himawari_cod/'

In [4]:
Re = 6371e3

target_lat = np.arange(-48.787998, -6.83179855, 0.01979828)
target_lon = np.arange(107.51999664, 158.98019409, 0.01979828)

In [5]:
def plot_ctt(data):
    crs0 = ccrs.PlateCarree(central_longitude=0)

    plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=crs0)

    [lons, lats] = np.meshgrid(data.lon, data.lat)

    ax.coastlines()
    ax.set_extent((target_lon[0], target_lon[-1], target_lat[0], target_lat[-1]), crs=crs0)
    gl = ax.gridlines(draw_labels=True, linestyle=':')
    gl.top_labels, gl.right_labels = False, False

    plt.pcolormesh(lons, lats, data, vmin=180, vmax=320, cmap='turbo')
    cb = plt.colorbar()
    cb.set_label('[K]')
    plt.title(f'Himawari-8 CTT: {data.start_time}')
    
    plt.savefig(out_path+f'himawari_ctt_{data.start_time}.png', dpi=200, bbox_inches='tight')
    
def plot_cod(data):
    crs0 = ccrs.PlateCarree(central_longitude=0)

    plt.figure(figsize=(12, 6))
    ax = plt.axes(projection=crs0)

    [lons, lats] = np.meshgrid(data.lon, data.lat)

    ax.coastlines()
    ax.set_extent((target_lon[0], target_lon[-1], target_lat[0], target_lat[-1]), crs=crs0)
    gl = ax.gridlines(draw_labels=True, linestyle=':')
    gl.top_labels, gl.right_labels = False, False

    plt.pcolormesh(lons, lats, data, vmin=0, vmax=10, cmap='turbo')
    cb = plt.colorbar()
    cb.set_label('[K]')
    plt.title(f'Himawari-8 COD: {data.start_time}')
    
    plt.savefig(out_path+f'himawari_cod_{data.start_time}.png', dpi=200, bbox_inches='tight')

In [6]:
%%time
# loop over files, create plot and save.

for day in day_range:
    
    print('day:', day)
    
    filenames = glob(f'/g/data/v46/lb5963/HIMAWARI/08_V46_SUMM_2021-22/L2/geocatL2.HIMAWARI-8.2022{day:03d}*.nc')

    for i, filename in enumerate(filenames):
        
        print('hour:', i)
        
        global_scene = Scene(reader='geocat', filenames=[filename]) #reader="seviri_l1b_hrit",reader='ahi_hsd', 
        
        global_scene.load([var])
        data_var = global_scene[var]

        data_var_lat = data_var.y / Re * (180/np.pi)
        #cosphi = np.cos(np.deg2rad(ctt_lat))
        data_var_lon = data_var.x / (Re) * (180/np.pi) + 140.7 # lon_0 # *cosphi

        data_var = data_var.assign_coords({"lon": ("x", data_var_lon.values)})
        data_var = data_var.assign_coords({"lat": ("y", data_var_lat.values)})
        data_var = data_var.swap_dims({"x": "lon", "y": "lat"})

        data_var_interp = data_var.interp(lat=target_lat, lon=target_lon, method="linear") #nearest

        plot_cod(data_var_interp)

        del global_scene, data_var

In [ ]:
assert False

In [ ]:
%%time
filenames = ['/g/data/v46/lb5963/HIMAWARI/08_V46_SUMM_2021-22/L2/geocatL2.HIMAWARI-8.2022053.000000.FLDK.R20.nc']
#filenames = ['/g/data/v46/lb5963/HIMAWARI/08_V46_SUMM_2021-22/L1/geocatL1.HIMAWARI-8.2022053.000000.FLDK.R20.nc']

global_scene = Scene(reader='geocat', filenames=filenames) #reader="seviri_l1b_hrit",reader='ahi_hsd', 
 

In [ ]:
global_scene.available_dataset_names()

In [ ]:
%%time
global_scene.load([var])
ctt = global_scene[var]
ctt

In [ ]:
ctt.crs

In [ ]:
ctt.attrs['area']

In [ ]:
# convert from x/y to lat/lon

Re = 6371e3

ctt_lat = ctt.y / Re * (180/np.pi)

cosphi = np.cos(np.deg2rad(ctt_lat))

ctt_lon = ctt.x / (Re) * (180/np.pi) + 140.7 # lon_0 # *cosphi
ctt_lon

In [ ]:
# swap dims to lat/lon
ctt = ctt.assign_coords({"lon": ("x", ctt_lon.values)})
ctt = ctt.assign_coords({"lat": ("y", ctt_lat.values)})
ctt = ctt.swap_dims({"x": "lon", "y": "lat"})
ctt

In [ ]:
# %%time
# [lons, lats] = np.meshgrid(ctt.lon, ctt.lat)

# plt.pcolormesh(lons, lats, ctt)

In [ ]:
# define a regular grid

target_lat = np.arange(-48.787998, -6.83179855, 0.01979828)
target_lon = np.arange(107.51999664, 158.98019409, 0.01979828)

In [ ]:
# bilinear interpolation to regular grid

ctt_interp = ctt.interp(lat=target_lat, lon=target_lon, method="linear") #nearest
ctt_interp

In [ ]:
global_scene.show('ACHA_mode_8_cloud_top_temperature')